In [ ]:
%load_ext autoreload
%autoreload 2


import random

import numpy as np
import matplotlib.pyplot as plt

import glob
import os

from autoscript_sdb_microscope_client.structures import AdornedImage

from liftout.fibsem import calibration, utils
from liftout import utils as l_utils 
from liftout.detection import detection

from pprint import pprint

import scipy.ndimage as ndi
import skimage

from liftout.detection.DetectionModel import DetectionModel
from liftout.detection.detection import DetectionType
from liftout.detection import utils as det_utils

from liftout.fibsem.structures import Point



In [ ]:
# TODO:
# cosine stretch
# titlted = less pixels for same info


In [ ]:
sample = "dm*"
_id = "*b0*"
img_type = "_*"
search = "landing_needle_land_sample_lowres"
filenames = glob.glob(f"data/{sample}/{_id}/*{search}{img_type}.tif") 
random.shuffle(filenames)

print(f"{len(filenames)} selected.")

### Land Lamella on Post

In [ ]:
# Step
## align_lamella_to_centre_landing_post_ib

sample = "**"
_id = "*"
img_type = "_ib"
search = "landing_needle_land_sample_*"
filenames = glob.glob(f"data/{sample}/{_id}/*{search}{img_type}.tif") 
random.shuffle(filenames)

weights_file = r"C:\Users\pcle0002\Documents\repos\autoliftout\liftout\model\models\boost_n05_model.pt"
model = DetectionModel(weights_file=weights_file)

print(f"{len(filenames)} selected.")

for fname in filenames:
    img = AdornedImage.load(fname)
    
    print(f"{os.path.basename(fname)}, beam: {img.metadata.acquisition.beam_type}")
  
    det = detection.locate_shift_between_features(model, img, shift_type=(DetectionType.LamellaEdge, DetectionType.LandingPost))
    # det.display_image = None

    # fig, ax = plt.subplots(1, 4, figsize=(35, 35))
    # ax[0].imshow(img.data, cmap="gray")
    # ax[0].set_title("raw image")
    # ax[1].imshow(mask, cmap="gray")
    # ax[1].set_title("mask")
    # ax[2].imshow(edge_mask, cmap="turbo")
    # ax[2].set_title("masked edge detection")
    # ax[-1].imshow(det.adorned_image.data, cmap="gray")
    # ax[-1].plot([0, pt.x], [pt.y, pt.y], "w--", ms=50, markeredgewidth=2)
    # ax[-1].plot([pt.x, pt.x], [pt.y, img.data.shape[0]], "w--", ms=50, markeredgewidth=2)
    # ax[-1].plot(det.features[0].feature_px.x, det.features[0].feature_px.y, "r+", ms=50, markeredgewidth=2)
    # ax[-1].plot(det.features[1].feature_px.x, det.features[1].feature_px.y, "c+", ms=50, markeredgewidth=2)
    # ax[-1].set_title("detection image")
    # plt.show()

    fig = det_utils.plot_detection_result(det)
    plt.show()
    plt.close(fig)


    # TODO: landing post detections improvements... mask a better area



### Needle Insertion Entry Alignment

In [ ]:

## TODO: BETTER LANDING ENTRY

# 1. take image before inserting
# 2. insert needle lower in eb
# 3. take difference between two images
# 4. detect needle position from difference image
# 5. detect landing post from centroid


# only use if we cant make the needle insert lower into the void area
# if we cant 

from fileinput import filename


filenames = []

## landing
sample = "dm*"
_id = "*"

img_type = "label"
search = "landing_needle_land_sample_lowres_"
label_filenames = glob.glob(f"data/{sample}/{_id}/*{search}{img_type}.tif") 
random.shuffle(filenames)


for fname in label_filenames:

    base = os.path.dirname(fname)
    lower_fname = glob.glob(f"{base}/{search}eb.tif")[0]
    filenames.append((fname, lower_fname))


# liftout
sample = "*"
_id = "*"
img_type = "_*b"
search = "needle_liftout_initial_position"
label_filenames2 = glob.glob(f"data/{sample}/{_id}/*{search}{img_type}.tif") 


for fname in label_filenames2:

    base = os.path.dirname(fname)
    img_type = "eb" if "eb" in os.path.basename(fname) else "ib"
    search = "needle_liftout_start_position_lowres_"
    lower_fname = glob.glob(f"{base}/{search}{img_type}.tif")[0]

    filenames.append((fname, lower_fname))

## Charging a problem
# - take images to drain the charge?

weights_file = r"C:\Users\pcle0002\Documents\repos\autoliftout\liftout\model\models\boost_n05_model.pt"
model = DetectionModel(weights_file=weights_file)

print(f"{len(filenames)} filenames")

for fname in filenames:

    pprint(fname)

    ref_image = AdornedImage.load(fname[0])
    new_image = AdornedImage.load(fname[1])

    # needle tip detection
    needle = detection.detect_needle_tip_v2(ref_image, new_image)   # right most edge

    if "landing" in fname[0]:
        det = detection.locate_shift_between_features_v2(new_image, ref_image, shift_type=(DetectionType.NeedleTip, DetectionType.LandingPost))
        det.display_image = ref_image.data
    
    if "liftout" in fname[0]:
        det = detection.locate_shift_between_features_v2(new_image, ref_image, shift_type=(DetectionType.NeedleTip, DetectionType.ImageCentre))
        det.display_image = new_image.data
    
    det.features[0].feature_px = needle
    # fig, ax  = plt.subplots(1, 3, figsize=(30, 30))

    # ax[0].imshow(img1.data, cmap="gray")
    # ax[0].set_title("Initial Entry Point")
    # ax[1].imshow(img2.data, cmap="gray")
    # ax[1].set_title("After Y Movement")
    # ax[2].imshow(img2.data, cmap="gray")
    # ax[2].set_title("Needle Tip Detection")
    # ax[2].plot(det.features[0].feature_px.x, det.features[0].feature_px.y, color="lime", marker="+", ms=50, markeredgewidth=2)
    # ax[2].plot(det.features[1].feature_px.x, det.features[1].feature_px.y, color="white", marker="+", ms=50, markeredgewidth=2)
    # plt.show()

    pprint(det.distance_metres)

    fig = det_utils.plot_detection_result(det)
    plt.show()
    plt.close(fig)

In [ ]:
# TODO: START_HERE
# need to track the needle as it moves?
# get postion -> get movement -> new position = initial guess

# how to incorporate the difference images into hte workflow
# refactor locate_shift_between_features_v2 (no model)
#       - use ref_image? could be useful for lamella centre as well..
#       - use a reference for detecting the lamella centre

stage, movement,goal, rate, note, implemented,step

1. util, eucentric_correction, correct the eucentric alignment for both beams
2. trench, align_to_ref_lamella, align the image to the  lamella position selected
3. jcut, align_to_ref_trench, align the image to the lamella trench reference image
4. jcut, align_to_ref_trench_tilt, align the image to the lamella trench reference image after tilting to jcut angle
5. jcut, align_to_ref_jcut_tilt, align the image tot the jcut reference after tilting back to flat to ebeam,
6. liftout, align_to_ref_jcut, align the image to the jcut reference image (flat to ebeam)
7. liftout, align_needle_to_lamella_centre_eb, align the needle to the centre of the lamella (eb, xy)
8. liftout, land_needle_on_lamella_centre_ib, move the needle down until contact is made with the lamella (detect brightness change) 
9. liftout, align_jcut_sever_to_lamella, align the jcut severing cut based on the image centre and lamella size
10. landing, align_needle_to_landing_post_centre_eb, "align the needle to the centre of the landing post (eb, xy)
11. landing, check_lamella_on_needle_ib, validate whether the lamella is attached to the needle
12. landing, land_lamella_on_post_ib, move the needle (x) until it makes contact with the landing post (edges overlap?)
13. landing, align_weld_to_post, align the weld to the intersection of the landing post and the lamella edge
14. landing, align_needle_cut_ib, align the pattern to cut the needle from the lamella
15. reset, align_needle_to_image_centre_ib, align the needle to the centre of the image 
16. thin, align_lamella_centre_to_image_centre_ib, align the centre of the lamella to the centre of the image
17. polsih, align_to_ref_thin_lamella, align the image to the thin lamella reference image